# Example Jupyter Notebook for micropattern (MP) image analysis

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import imageio.v3 as iio
import os

from support_functions import *
from micropattern_analysis import *

## Introduction to image analysis of MPs

In [ ]:
img_file = r"D:\OneDrive - Universität Wien\Adameyko Lab\FS3_MP\fluorescence_images\all_fates_pax6_isi12_sox10\MP_ISTA_STAN_BMP4_800um_DAPI_pax6_555_sox10_647_isi12_488_2024_01_05__17_57_15.lsm"

image = iio.imread(img_file)

Here we can see that the images are stored as numpy arrays with multiple dimensions. Let's quickly break it down how the images are stored in a ```numpy``` matrix

In [ ]:
print_image_dim_explanation()

As we can see, our image is a multichannel image with multiple z-stacks. Where the first axis (dimension) corresponds to each z-stack with its corresponding channels. The second axis describes the different channels found in the image. The last two axis are responsible for storage of the pixel values of each channel and corresponding z-stack.

Now, as we have seen our images contain z-stacks for which we are not interested in for our analysis. Therefore we can maximise the images to a single stack.

In [ ]:
shape_old = image.shape
shape_new = maximise_and_normalize(image).shape
print(f'Original shape\tvs.\tnew shape')
print(f'{shape_old}\t->\t{shape_new}')

As we can see this effectively reduced one image dimension. And we can use this to override our current image.

In [ ]:
image = maximise_and_normalize(image)

We can also visualize this through plotting all the images and stacks of the image and be able see the difference.

In [ ]:
# TODO: create function to plot all stacks and/or channels of one image

Now we can start with the actual analysis. Here the following flow-diagram can be used to analyze MPs.

In [ ]:
# TODO: plot flow diagram of MP analysis

TODO: add additional supporting information to diagram 

As the first step of analysis is to create a mask of the image by using the DAPI channel or any other which seems appropriate, we need to first decide on which method to choose. Here the general idea is to find a threshold and apply it to the channel of our choice (i.e. DAPI). In the toolbox we already have we can use an automatic threshold algorithm. Otsu's algorithm provides a good basis for our analysis and also can provide us with more than one thresholds. This can be useful if we want to exclude the background of our image. Below we can apply a Multi-Otsu algorithm to find multiple thresholds and compare their outputs.   

In [ ]:
"""
    Plot Multi-Otsu's threshold for our image with 4 different classes, which results in 3
    thresholds
"""
plot_multi_otsu_thresholds(image, 4)

In [ ]:
# TODO: continue with introductory notebook sections

## Streamlined image analysis through the usage of pipelines

First we have to select a directory from which the images should be accessed. Here we could first set the directory - where our images are located - to our current working directory. This gives us the opportunity to expect the subdirectories or files which we may want to include or exclude from our analysis.

In [ ]:
dir = r"D:\OneDrive - Universität Wien\Adameyko Lab\FS3_MP\fluorescence_images"
os.chdir(dir)
dirs = os.listdir(".")
print(dirs)

Here the selected directory contains some subdirectories which we are not interested in including. Therefore we can select specifically which directories to include into the list of paths.

In [ ]:
dirs = dirs[2:4]
print(dirs)

Much better! With this done we can start loading our images into the notebook. For this and further actions with first have to import the custom script for analysis.

In [ ]:
# TODO: put code up to introduction
image_dict = get_files_and_images(dirs)

In [ ]:
first_element_key = list(image_dict.keys())[0]
first_element_value = image_dict.get(first_element_key)
print(f'type of image_dict:\t{type(image_dict)}')
print(f'Number of images:\t{len(image_dict.keys())}')
print(f'First element of image_dict w/ values and types:')
print(f'\t"{first_element_key}"\t{type(first_element_key)}')
print(f'\t{first_element_value.shape}\t{type(first_element_value)}')

As we can see the method ```get_files_and_images()``` returns a dictionary with the image file names as keys and the images as a values. As seen above we can maximise the images and normalize them in one go.

If you want to "untangle" the images from their file names then following line can accomplish this. 

In [ ]:
file_names, images = zip(*image_dict.items())

And if you want to skip the ```image_dict``` step altogether, then we can directly call the ```get_files_and_images()``` function.

In [ ]:
file_names, images = zip(*get_files_and_images(dirs).items())

In [14]:
from toolz import pipe

dapi_num = 3

v_new = list(map(
    lambda x:
    pipe(
        x,
        lambda f: maximise_and_normalize(f),
        lambda f: apply_multiotsu_to_image(f, dapi_num),
        lambda f: apply_multiotsu_to_channel(f, 0)
    ),
    images)
)

print(f'Original shape:\t{images[0].shape}')
print(f'New shape:\t{v_new[0].shape}')

Original shape:	(3, 4, 1024, 1024)
New shape:	(4, 1024, 1024)
